Based off of:
https://langchain-ai.github.io/langgraph/tutorials/introduction/

Setup

In [1]:
import getpass
import os
import json

if "OPENAI_API_KEY" not in os.environ:
    os.environ["OPENAI_API_KEY"] = getpass.getpass("Enter your OpenAI API key: ")


if "TAVILY_API_KEY" not in os.environ:
    os.environ["TAVILY_API_KEY"] = getpass.getpass("Enter your Tavily API key: ")

Chatbot Init

In [27]:
from typing import Annotated

from typing_extensions import TypedDict

from langgraph.graph import StateGraph, START, END
from langgraph.graph.message import add_messages
from langgraph.prebuilt import ToolNode, tools_condition

from langchain_community.tools import TavilySearchResults

from langchain_openai import ChatOpenAI

from langgraph.checkpoint.memory import MemorySaver


class State(TypedDict):
    messages: Annotated[list, add_messages]


graph_builder = StateGraph(State)

memory = MemorySaver()

tool = TavilySearchResults(
    max_results=2,
    search_depth="advanced"
)

tools=[tool]

llm = ChatOpenAI(
    model="gpt-4o-mini-2024-07-18",
    max_tokens=500,
    timeout=None,
    max_retries=1,
    temperature=0
)

llm_with_tools = llm.bind_tools(tools)

def chatbot(state: State):
    return {"messages": [llm_with_tools.invoke(state["messages"])]}


graph_builder.add_node("chatbot", chatbot)

tool_node = ToolNode(tools=[tool])
graph_builder.add_node("tools", tool_node)

graph_builder.add_conditional_edges(
    "chatbot",
    tools_condition,
)
graph_builder.add_edge("tools", "chatbot")
graph_builder.set_entry_point("chatbot")
graph_builder.add_edge("chatbot", END)

#graph = graph_builder.compile(checkpointer=memory)
graph = graph_builder.compile()


Actual Chatbot

In [ ]:
def stream_graph_updates(user_input: str):
    #config = {"configurable": {"thread_id": "1"}}

    #events = graph.stream({"messages": [("user", user_input)]}, config=config, stream_mode = "values")
    events = graph.stream({"messages": [("user", user_input)]})
    for event in events:
        for value in event.values():
            print("Assistant: ", value["messages"][-1].content)


while True:
    try:
        user_input = input("User: ")
        if user_input.lower() in ["quit", "exit", "q"]:
            print("Goodbye!")
            break
        print("User: ", user_input)
        stream_graph_updates(user_input)
        
    except KeyboardInterrupt:
        print("\nGoodbye!")
        break